In [1]:
import torch 
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder

In [ ]:
#train and test data directory
data_dir = "./seg_train/seg_train/"
test_data_dir = "./seg_test/seg_test/"

# load the train and test data 
dataset = ImageFolder(data_dir,transform=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor()
]))